In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [ ]:
from asyncio import run, sleep

from asyncssh import connect

In [ ]:
# less nesting
from contextlib import AsyncExitStack
from pathlib import Path
from typing import AsyncGenerator


HPC_USER = ...
HPC_HOST = ...


class LocalHPCClient:
    def __init__(self, host: str, user: str):
        self.host = host
        self.user = user

    async def scp(
        self,
        source_stream: AsyncGenerator[bytes, None],
        target_path: Path,
    ):
        async with AsyncExitStack() as stack:
            conn = await stack.enter_async_context(
                connect(self.host, username=self.user)
            )
            sftp = await stack.enter_async_context(conn.start_sftp_client())
            file = await stack.enter_async_context(sftp.open(str(target_path), 'wb'))

            async for chunk in source_stream:
                await file.write(chunk)

In [ ]:
class PBSClient:
    def __init__(self, host: str, user: str):
        self.host = host
        self.user = user

    async def queue_state(self):
        # qstat
        async with connect(self.host, username=self.user) as connection:
            result = await connection.run('qstat', check=True)
            stdout = result.stdout.strip()

        return stdout

    async def queue_submit(self):
        # qsub
        pass

    async def queue_delete(self):
        # qdel
        pass

    async def queue_hold(self):
        # qhold
        pass

    async def queue_release(self):
        # qrls
        pass

    async def trace_job(self):
        # tracejob
        pass

In [ ]:
from enum import Enum


class PBSJobState(str, Enum):
    """
    PBS Pro 2022.1.1 Job States

    Reference:
        https://help.altair.com/2022.1.0/PBS%20Professional/PBSReferenceGuide2022.1.pdf
    """

    B = 'begun'
    """Job arrays only (7): at least one subjob has started."""

    E = 'exiting'
    """(5): Job is exiting after having run."""

    F = 'finished'
    """(9): Job has completed execution, failed during execution, or was deleted."""

    H = 'held'
    """(2): Job is held by user, admin, or server."""

    M = 'moved'
    """(8): Job was moved to another server."""

    Q = 'queued'
    """(1): Job is queued, eligible to run or be routed."""

    R = 'running'
    """(4): Job is currently running."""

    S = 'suspended'
    """(400): Job is suspended by scheduler to free up resources."""

    T = 'transition'
    """(0): Job is transitioning to or from a server."""

    U = 'unknown'
    """(410): Job is suspended due to workstation becoming busy."""

    W = 'waiting'
    """(3): Job is waiting for its execution time or delayed due to stagein failure."""

    X = 'exited'
    """(6): Subjobs only; subjob is finished or expired."""